## Importing Necessary Libraries

Importing the essential libraries to build the **Streamlit** web application, load the trained models, connect to a database, and handle data.

---

### Libraries Used:

1. **`streamlit` (as `st`):**  
   Streamlit is used to build interactive web applications for machine learning models. It allows easy integration of machine learning models and real-time prediction.

2. **`numpy` (as `np`):**  
   Numpy is a fundamental library for numerical computing in Python. It is often used for handling arrays and mathematical operations.

3. **`joblib`:**  
   Joblib is used to load the pre-trained models saved earlier as `.pkl` files. It allows for efficient model loading and saving.

4. **`sqlite3`:**  
   SQLite3 is used to connect to SQLite databases, allowing for easy storage and retrieval of data in the app.

5. **`pandas` (as `pd`):**  
   Pandas is used for data manipulation and analysis, making it easy to handle tabular data (e.g., CSV files, SQL queries).


In [1]:
import streamlit as st
import numpy as np
import joblib
import sqlite3
import pandas as pd

##  Load Trained Models and Scaler

To make real-time predictions, we need to load the pre-trained machine learning models and the scaler that were saved earlier. This allows us to apply the trained models to new data.

---  
This code loads the saved models and scaler from disk using `joblib`. These pre-trained models are then ready to be used for predictions in the Streamlit app.

---

In [2]:
# Load trained models and scaler
lr_model = joblib.load(r"D:\loan_approval_pred\ml_model\lr_model.pkl")
dt_model = joblib.load(r"D:\loan_approval_pred\ml_model\dtree_model.pkl")
rf_model = joblib.load(r"D:\loan_approval_pred\ml_model\rf_model.pkl")
scaler = joblib.load(r"D:\loan_approval_pred\ml_model\scaler.pkl")

##  SQLite3 Setup

This code sets up the connection to an SQLite database (`loan_predictions.db`) and creates a cursor object to interact with the database.

---

1. **Establish Connection (`conn`):**  
   The `sqlite3.connect()` function is used to connect to the SQLite database. If the database does not exist, it will be created in the current working directory.

2. **Create Cursor (`cursor`):**  
   The `cursor()` method creates a cursor object that allows us to execute SQL commands (e.g., SELECT, INSERT) and interact with the database.


In [3]:
# SQLite3 setup
conn = sqlite3.connect('loan_predictions.db')
cursor = conn.cursor()

##  Create Table if It Doesn't Exist
  This code creates a table called `LoanApplications` in the SQLite database (`loan_predictions.db`) if it does not already exist. The table will store the necessary features for each loan application, along with the predicted loan status.

---

### Table Structure:

1. **`id` (INTEGER PRIMARY KEY AUTOINCREMENT):**  
   A unique identifier for each loan application. This field will automatically increment as new records are inserted.

2. **`cibil_score` (REAL):**  
   The CIBIL score of the applicant (a numeric value).

3. **`loan_term` (INTEGER):**  
   The duration of the loan (in years or months, depending on the input).

4. **`income_annum` (REAL):**  
   The applicant’s annual income (numeric value).

5. **`no_of_dependents` (INTEGER):**  
   The number of dependents of the loan applicant.

6. **`loan_amount` (REAL):**  
   The total loan amount being requested.

7. **`luxury_assets_value` (REAL):**  
   The value of luxury assets owned by the applicant (if applicable).

8. **`predicted_status` (TEXT):**  
   The predicted loan approval status, which will be either "Approved" or "Rejected" (this will be filled after model predictions).

In [4]:
# Create table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS LoanApplications (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    cibil_score REAL,
    loan_term INTEGER,
    income_annum REAL,
    no_of_dependents INTEGER,
    loan_amount REAL,
    luxury_assets_value REAL,
    predicted_status TEXT
)
""")
conn.commit()

##  Streamlit Page Configuration and Title

This step configures the layout and title of the Streamlit web application, providing the user with a clean and intuitive interface for input.

---

### Steps:

1. **Set Page Configuration (`st.set_page_config()`):**  
   The `set_page_config()` function sets the page title and layout. 
   - `page_title="Loan Approval Predictor"` sets the title that appears in the browser tab.
   - `layout="centered"` ensures that the app content is centered on the page, providing a neat and user-friendly appearance.

2. **Set Title (`st.title()`):**  
   The `st.title()` function displays the title of the app on the page. In this case, it is set to **"Loan Approval Prediction App"**.

3. **Write Instructions (`st.write()`):**  
   The `st.write()` function is used to display additional text. Here, it prompts the user to enter their loan application details below the title.

In [5]:
st.set_page_config(page_title="Loan Approval Predictor", layout="centered")
st.title("Loan Approval Prediction App")
st.write("Enter your loan application details below:")

2025-04-20 12:22:21.876 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:22:21.877 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:22:22.448 
  command:

    streamlit run D:\python\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-20 12:22:22.449 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:22:22.450 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:22:22.451 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:22:22.452 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mod

##  User Input Collection

This step gathers the necessary input from the user via a series of form fields to predict loan approval based on their application details.

--- 
This code creates input fields for the user to provide their loan application details, which will be used for predictions. The inputs include numeric and categorical values like the CIBIL score, loan amount, marital status, etc.

In [6]:
# User input
cibil_score = st.number_input("CIBIL Score", min_value=0.0, max_value=900.0, value=750.0)
loan_term = st.number_input("Loan Term (Months)", min_value=1, max_value=360, value=60)
income_annum = st.number_input("Annual Income (₹)", min_value=0.0, value=500000.0)
no_of_dependents = st.number_input("Number of Dependents", min_value=0, max_value=10, value=1)
loan_amount = st.number_input("Loan Amount (₹)", min_value=0.0, value=300000.0)
luxury_assets_value = st.number_input("Luxury Assets Value (₹)", min_value=0.0, value=1000000.0)
gender = st.selectbox("Gender (1-Male,0-Female)", [1,0])
married = st.selectbox("Married (1-Yes, 0-No)", [1,0])
education = st.selectbox("Education(College Graduate)(1-Yes, 0-No)", [1,0])
self_employed = st.selectbox("Self Employed (1-Yes, 0-No)", [1,0])

2025-04-20 12:24:11.130 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:24:11.131 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:24:11.132 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:24:11.133 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:24:11.133 Session state does not function when running a script without `streamlit run`
2025-04-20 12:24:11.134 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:24:11.135 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:24:11.135 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:24

## Loan Prediction and Result Display

---
This code predicts whether a loan will be approved or rejected based on the user-provided input and displays the results. It also saves the prediction details into an SQLite database.

---

In [7]:
if st.button("Predict Loan Status"):
    # Prepare the full 10-feature input
    input_data = np.array([[no_of_dependents, income_annum, loan_amount, loan_term, cibil_score, luxury_assets_value,
                            gender, married, education, self_employed]])

    # Scale all 10 inputs (as expected by your scaler)
    scaled_input = scaler.transform(input_data)

    # Slice only the first 6 features for models that were trained on 6 inputs
    scaled_input_for_model = scaled_input[:, :6]

    # Predictions
    lr_pred = lr_model.predict(scaled_input[:, :6])[0]
    dt_pred = dt_model.predict(scaled_input[:, :6])[0]
    rf_pred = rf_model.predict(scaled_input[:, :6])[0]


    # Majority vote logic
    votes = [lr_pred, dt_pred, rf_pred].count(1)
    final_status = "Approved" if votes >= 2 else "Rejected"

    # Show result to user
    st.subheader("Prediction Results:")
    st.write(f"Logistic Regression: {' Approved' if lr_pred == 1 else ' Rejected'}")
    st.write(f"Decision Tree: {' Approved' if dt_pred == 1 else ' Rejected'}")
    st.write(f"Random Forest: {' Approved' if rf_pred == 1 else ' Rejected'}")

    if final_status == "Approved":
        st.success(f" Majority Decision: Loan {final_status}!")
    else:
        st.error(f" Majority Decision: Loan {final_status}.")

    # Insert prediction into SQLite
    cursor.execute("""
    INSERT INTO LoanApplications (cibil_score, loan_term, income_annum, no_of_dependents, loan_amount, luxury_assets_value, predicted_status)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (cibil_score, loan_term, income_annum, no_of_dependents, loan_amount, luxury_assets_value, final_status))
    conn.commit()

    st.info(" Application data saved to database.")


2025-04-20 12:27:15.667 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:27:15.667 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:27:15.668 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:27:15.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:27:15.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


##  Viewing and Exporting Saved Loan Predictions

This section allows users to review past loan predictions and export the data for external use.

---

### 🔍 **View Previous Predictions**  
  When the "Show Saved Applications" button is clicked from the sidebar, this block fetches all records from the `LoanApplications` table in the SQLite database.
- **Functionality:**  
  - The saved loan applications are displayed in the sidebar.
  - The most recent applications appear at the top (`ORDER BY id DESC`).

---

### 📤 **Export Predictions to Excel**  
  Allows users to export the full list of loan predictions to an Excel file for reporting or analysis.
- **Functionality:**  
  - When the "Export to Excel" button is pressed, the application reads all data from the `LoanApplications` table into a pandas DataFrame.
  - The data is saved as an Excel file named `LoanApplications.xlsx` in the working directory.
  - After saving, a success message is shown in the sidebar.



In [8]:
# View previous predictions
if st.sidebar.button(" Show Saved Applications"):
    cursor.execute("SELECT * FROM LoanApplications ORDER BY id DESC")
    records = cursor.fetchall()
    st.sidebar.write("### Saved Applications")
    for record in records:
        st.sidebar.write(record)
# Sidebar: Export to Excel
if st.sidebar.button(" Export to Excel"):
    df = pd.read_sql_query("SELECT * FROM LoanApplications", conn)
    df.to_excel('LoanApplications.xlsx', index=False)
    st.sidebar.success(" Exported to LoanApplications.xlsx")

# Close DB connection on exit
conn.close()

2025-04-20 12:28:43.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:28:43.828 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:28:43.828 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:28:43.828 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:28:43.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:28:43.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:28:43.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-20 12:28:43.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

## Run the two lines in Command Prompt to open a webpage

In [ ]:
#cd /d D:\loan_approval_pred
#streamlit run loan_ui.py
